## OBS:
This notebook is only slightly edited from Zebedee Nicholls notebook, see [here](https://gitlab.com/rcmip/rcmip/-/blob/master/notebooks/results/phase-1/database-generation.ipynb)

In [1]:
from ar6_ch6_rcmipfigs.constants import INPUT_DATA_DIR

__depends__ = []
__dest__ = [INPUT_DATA_DIR+
    "/data/database-results/phase-1/timestamp.txt",
    INPUT_DATA_DIR+"/data/database-observations/timestamp.txt",
]

/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs
/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in


In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

# Database generation


In this notebook we process the data into a database we can later query to make plots/do analysis etc.

## Imports

In [3]:
import logging
import os.path
import re
from pathlib import Path
from distutils.util import strtobool

import pandas as pd
import tqdm
from scmdata import ScmDataFrame, df_append

<IPython.core.display.Javascript object>

pyam - INFO: Running in a notebook, setting `pyam` logging level to `logging.INFO` and adding stderr handler


<IPython.core.display.Javascript object>

In [4]:
from ar6_ch6_rcmipfigs.utils.database_generation import check_all_variables_and_units_as_in_protocol, \
    check_all_scenarios_as_in_protocol, unify_units, save_into_database, mce_get_quantile, hector_get_quantile
from ar6_ch6_rcmipfigs.utils.database_generation2 import get_res_fl, read_db, fix_ssp370_names, _assert, \
    fix_hector_mce_mislab, fix_quant_FaIR, fix_quant_wasp, unify_units_check_names

TEST_RUN = strtobool(os.getenv("CI", "False")) or False
TEST_RUN

False

<IPython.core.display.Javascript object>

In [5]:
logger = logging.getLogger()

<IPython.core.display.Javascript object>

## Constants

In [10]:
from ar6_ch6_rcmipfigs.constants import  INPUT_DATA_DIR
OUTPUT_DATABASE_PATH = os.path.join(INPUT_DATA_DIR, "database-results", "phase-1/")

OBS_DATABASE_PATH = os.path.join(INPUT_DATA_DIR, "database-observations/")

<IPython.core.display.Javascript object>

In [11]:
from ar6_ch6_rcmipfigs.utils.misc_func import make_folders

if not os.path.isdir(OUTPUT_DATABASE_PATH):
    make_folders(OUTPUT_DATABASE_PATH)

if not os.path.isdir(OBS_DATABASE_PATH):
    make_folders(OBS_DATABASE_PATH)

<IPython.core.display.Javascript object>

## Protocol

In [12]:
SCENARIO_PROTOCOL = os.path.join(INPUT_DATA_DIR, "data", "protocol", "rcmip-emissions-annual-means.csv"
)

<IPython.core.display.Javascript object>

In [13]:
protocol_db = ScmDataFrame(SCENARIO_PROTOCOL)
protocol_db.head()

time                                                                               1750-01-01 00:00:00  \
model scenario region variable                   unit      activity_id    mip_era                        
AIM   rcp60    World  Emissions|BC               Mt BC/yr  not_applicable CMIP5                    NaN   
                      Emissions|CH4              Mt CH4/yr not_applicable CMIP5                    NaN   
                      Emissions|CO               Mt CO/yr  not_applicable CMIP5                    NaN   
                      Emissions|CO2              Mt CO2/yr not_applicable CMIP5                    NaN   
                      Emissions|CO2|MAGICC AFOLU Mt CO2/yr not_applicable CMIP5                    NaN   

time                                                                               1751-01-01 00:00:00  \
model scenario region variable                   unit      activity_id    mip_era                        
AIM   rcp60    World  Emissions|BC               Mt BC/yr  not_applicable CMIP5                    NaN   
                      Emissions|CH4              Mt CH4/yr not_applicable CMIP5                    NaN   
                      Emissions|CO               Mt CO/yr  not_applicable CMIP5                    NaN   
                      Emissions|CO2              Mt CO2/yr not_applicable CMIP5                    NaN   
                      Emissions|CO2|MAGICC AFOLU Mt CO2/yr not_applicable CMIP5                    NaN   

time                                                                               1752-01-01 00:00:00  \
model scenario region variable                   unit      activity_id    mip_era                        
AIM   rcp60    World  Emissions|BC               Mt BC/yr  not_applicable CMIP5                    NaN   
                      Emissions|CH4              Mt CH4/yr not_applicable CMIP5                    NaN   
                      Emissions|CO               Mt CO/yr  not_applicable CMIP5                    NaN   
                      Emissions|CO2              Mt CO2/yr not_applicable CMIP5                    NaN   
                      Emissions|CO2|MAGICC AFOLU Mt CO2/yr not_applicable CMIP5                    NaN   

time                                                                               1753-01-01 00:00:00  \
model scenario region variable                   unit      activity_id    mip_era                        
AIM   rcp60    World  Emissions|BC               Mt BC/yr  not_applicable CMIP5                    NaN   
                      Emissions|CH4              Mt CH4/yr not_applicable CMIP5                    NaN   
                      Emissions|CO               Mt CO/yr  not_applicable CMIP5                    NaN   
                      Emissions|CO2              Mt CO2/yr not_applicable CMIP5                    NaN   
                      Emissions|CO2|MAGICC AFOLU Mt CO2/yr not_applicable CMIP5                    NaN   

time                                                                               1754-01-01 00:00:00  \
model scenario region variable                   unit      activity_id    mip_era                        
AIM   rcp60    World  Emissions|BC               Mt BC/yr  not_applicable CMIP5                    NaN   
                      Emissions|CH4              Mt CH4/yr not_applicable CMIP5                    NaN   
                      Emissions|CO               Mt CO/yr  not_applicable CMIP5                    NaN   
                      Emissions|CO2              Mt CO2/yr not_applicable CMIP5                    NaN   
                      Emissions|CO2|MAGICC AFOLU Mt CO2/yr not_applicable CMIP5                    NaN   

time                                                                               1755-01-01 00:00:00  \
model scenario region variable                   unit      activity_id    mip_era                        
AIM   rcp60    World  Emissions|BC               Mt BC/yr  not_applicable

<IPython.core.display.Javascript object>

In [14]:
protocol_db["scenario"].unique()

array(['rcp60', 'ssp370', 'ssp370-lowNTCF-aerchemmip',
       'ssp370-lowNTCF-gidden', 'historical', 'ssp434', 'ssp460', 'rcp26',
       'ssp119', 'ssp126', 'rcp85', 'ssp245', 'rcp45', 'ssp534-over',
       'ssp585', 'esm-bell-1000PgC', 'esm-bell-2000PgC',
       'esm-bell-750PgC', 'esm-pi-CO2pulse', 'esm-pi-cdr-pulse',
       'esm-piControl', 'historical-cmip5'], dtype=object)

<IPython.core.display.Javascript object>

In [15]:
DATA_PROTOCOL = os.path.join(INPUT_DATA_DIR,
    "data",
    "submission-template",
    "rcmip-data-submission-template.xlsx",
)

<IPython.core.display.Javascript object>

In [16]:
protocol_variables = pd.read_excel(DATA_PROTOCOL, sheet_name="variable_definitions")
protocol_variables.columns = protocol_variables.columns.str.lower()
protocol_variables.head()

,idx,category,variable,unit,definition,tier
0,1,Atmospheric Concentrations,Atmospheric Concentrations|CH4,ppb,atmospheric concentrations of CH4,1
1,2,Atmospheric Concentrations,Atmospheric Concentrations|CO2,ppm,atmospheric concentrations of CO2,1
2,3,Atmospheric Concentrations,Atmospheric Concentrations|F-Gases,ppm,equivalent species atmospheric concentrations ...,3
3,4,Atmospheric Concentrations,Atmospheric Concentrations|F-Gases|HFC,ppm,equivalent species atmospheric concentrations ...,3
4,5,Atmospheric Concentrations,Atmospheric Concentrations|F-Gases|HFC|HFC125,ppt,atmospheric concentrations of HFC125,2


<IPython.core.display.Javascript object>

In [17]:
protocol_scenarios = pd.read_excel(
    DATA_PROTOCOL, sheet_name="scenario_info", skip_rows=2
)
protocol_scenarios.columns = protocol_scenarios.columns.str.lower()
protocol_scenarios.head()

,unnamed: 0,scenario,description of scenario,detailed scenario description,priority
0,Instructions/Details,Scenario identifier used in the CMIP6 modeling...,Brief description of scenario,Documentation on es-doc + some other clarifyin...,"Top priority is Tier 1, additional runs are Ti..."
1,Answers,# Scenario ID,# Scenario Description,#Scenario Specification,# Tier in RCMP
2,NaN,piControl,pre-industrial control simulation (i.e. consta...,groups should run as long as is sensible/they ...,1
3,NaN,esm-piControl,pre-industrial control simulation with zero an...,should be identical to piControl except CO2 is...,1
4,NaN,esm-piControl-allGHG,pre-industrial control simulation with zero an...,should be identical to piControl except all GH...,2


<IPython.core.display.Javascript object>

## Model output

In [18]:
RESULTS_PATH = os.path.join(INPUT_DATA_DIR, "data", "results", "phase-1")






<IPython.core.display.Javascript object>

In [19]:
model_of_interest = [
#    ".*acc2.*v2-0-1.*",
#    ".*rcmip_phase-1_cicero-scm.*v5-0-0.*",
#    ".*escimo.*v2-0-1.*",
#    ".*fair-1.5-default.*v1-0-1.csv",
#    ".*fair-1.5-ens.*v1-0-1.csv",
#    ".*rcmip_phase-1_gir.*",
#    ".*greb.*v2-0-0.*",
#    ".*hector.*v2-0-0.*",
#    ".*MAGICC7.1.0aX-rcmip-phase-1.*",
#    ".*rcmip_phase-1_magicc7.1.0.beta.*_v1-0-0.*",
#    ".*rcmip_phase-1_magicc7.1.0.beta.*_v1-0-0.*",
#    ".*MAGICC7.1.0aX.*",
#    ".*mce.*v2-0-1.*",
#    ".*oscar-v3-0*v1-0-1.*",
#    ".*oscar-v3-0.*v1-0-1.*"
#    ".*wasp.*v1-0-1.*",
]

<IPython.core.display.Javascript object>

In [20]:
magic_base = ".*rcmip_phase-1_magicc7.1.0.beta.*_v1-0-0.*"
mag_rf = get_res_fl(mod)
l_magicc_mods = []
for _a in mag_rf:
    print(_a.split('/')[-1])
    l_magicc_mods.append('.*'+_a.split('/')[-1])

rcmip_phase-1_magicc7.1.0.beta-ipsl-cm6a-lr-r1i1p1f2_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-canesm5-r1i1p2f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-miroc6-r1i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-giss-e2-2-g-r1i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-bcc-esm1-r1i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-cesm2-r1i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-norcpm1-r1i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-canesm5-r10i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-noresm2-lm-r1i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-cnrm-cm6-1-hr-r1i1p1f2_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-giss-e2-1-h-r1i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-sam0-unicon-r1i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-cesm2-waccm-r1i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-e3sm-1-0-r1i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-ipsl-cm6a-lr-r10i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-fgoals

<IPython.core.display.Javascript object>

In [21]:
for mod in model_of_interest + l_magicc_mods:
    print(mod)
    results_files = get_res_fl(mod)
    print(results_files)
    db = read_db(results_files)
    db = fix_ssp370_names(db)
    _assert(db)
    db = fix_hector_mce_mislab(db)
    db = fix_quant_FaIR(db)
    db = fix_quant_wasp(db)
    clean_db = unify_units_check_names(db, protocol_variables, protocol_scenarios)
    save_into_database(clean_db, OUTPUT_DATABASE_PATH, "rcmip-phase-1")

.*rcmip_phase-1_magicc7.1.0.beta-ipsl-cm6a-lr-r1i1p1f2_v1-0-0.csv
['/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-ipsl-cm6a-lr-r1i1p1f2_v1-0-0.csv']


Filtered ScmDataFrame is empty!
Filtered ScmDataFrame is empty!
Filtered ScmDataFrame is empty!
Filtered ScmDataFrame is empty!


MAGICC7.1.0.beta_IPSL-CM6A-LR_R1I1P1F2
--------------------------------------



All clear for MAGICC7.1.0.beta_IPSL-CM6A-LR_R1I1P1F2





22 of 226 vars done for MAGICC7.1.0.beta_IPSL-CM6A-LR_R1I1P1F2, region World
44 of 226 vars done for MAGICC7.1.0.beta_IPSL-CM6A-LR_R1I1P1F2, region World
66 of 226 vars done for MAGICC7.1.0.beta_IPSL-CM6A-LR_R1I1P1F2, region World
88 of 226 vars done for MAGICC7.1.0.beta_IPSL-CM6A-LR_R1I1P1F2, region World
110 of 226 vars done for MAGICC7.1.0.beta_IPSL-CM6A-LR_R1I1P1F2, region World
132 of 226 vars done for MAGICC7.1.0.beta_IPSL-CM6A-LR_R1I1P1F2, region World
154 of 226 vars done for MAGICC7.1.0.beta_IPSL-CM6A-LR_R1I1P1F2, region World
176 of 226 vars done for MAGICC7.1.0.beta_IPSL-CM6A-LR_R1I1P1F2, region World
198 of 226 vars done for MAGICC7.1.0.beta_IPSL-CM6A-LR_R1I1P1F2, region World
220 of 226 vars done for MAGICC7.1.0.beta_IPSL-CM6A-LR_R1I1P1F2, region World
.*rcmip_phase-1_magicc7.1.0.beta-canesm5-r1i1p2f1_v1-0-0.csv
['/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-canesm5-r1i1p2f1_v1-0-0.csv

...

Filtered ScmDataFrame is empty!
Filtered ScmDataFrame is empty!
Filtered ScmDataFrame is empty!
Filtered ScmDataFrame is empty!


MAGICC7.1.0.beta_CANESM5_R1I1P1F1
---------------------------------



All clear for MAGICC7.1.0.beta_CANESM5_R1I1P1F1





22 of 226 vars done for MAGICC7.1.0.beta_CANESM5_R1I1P1F1, region World
44 of 226 vars done for MAGICC7.1.0.beta_CANESM5_R1I1P1F1, region World
66 of 226 vars done for MAGICC7.1.0.beta_CANESM5_R1I1P1F1, region World
88 of 226 vars done for MAGICC7.1.0.beta_CANESM5_R1I1P1F1, region World
110 of 226 vars done for MAGICC7.1.0.beta_CANESM5_R1I1P1F1, region World
132 of 226 vars done for MAGICC7.1.0.beta_CANESM5_R1I1P1F1, region World
154 of 226 vars done for MAGICC7.1.0.beta_CANESM5_R1I1P1F1, region World
176 of 226 vars done for MAGICC7.1.0.beta_CANESM5_R1I1P1F1, region World
198 of 226 vars done for MAGICC7.1.0.beta_CANESM5_R1I1P1F1, region World
220 of 226 vars done for MAGICC7.1.0.beta_CANESM5_R1I1P1F1, region World


<IPython.core.display.Javascript object>

In [22]:
a=['/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-ipsl-cm6a-lr-r1i1p1f2_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-canesm5-r1i1p2f1_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-miroc6-r1i1p1f1_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-giss-e2-2-g-r1i1p1f1_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-bcc-esm1-r1i1p1f1_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-cesm2-r1i1p1f1_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-norcpm1-r1i1p1f1_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-canesm5-r10i1p1f1_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-noresm2-lm-r1i1p1f1_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-cnrm-cm6-1-hr-r1i1p1f2_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-giss-e2-1-h-r1i1p1f1_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-sam0-unicon-r1i1p1f1_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-cesm2-waccm-r1i1p1f1_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-e3sm-1-0-r1i1p1f1_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-ipsl-cm6a-lr-r10i1p1f1_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-fgoals-g3-r1i1p1f1_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-ukesm1-0-ll-r1i1p1f2_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-mcm-ua-1-0-r1i1p1f2_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-bcc-csm2-mr-r1i1p1f1_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-giss-e2-1-g-r1i1p1f1_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-ipsl-cm6a-lr-r1i1p1f1_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-ec-earth3-veg-r1i1p1f1_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-miroc-es2l-r1i1p1f2_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-cnrm-esm2-1-r1i1p1f2_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-cnrm-cm6-1-r1i1p1f2_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-awi-cm-1-1-mr-r1i1p1f1_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-mpi-esm1-2-hr-r1i1p1f1_v1-0-0.csv', '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta-canesm5-r1i1p1f1_v1-0-0.csv']

<IPython.core.display.Javascript object>

In [23]:
l_magicc_mods = []
for _a in a:
    print(_a.split('/')[-1])
    l_magicc_mods.append('.*'+_a.split('/')[-1])

rcmip_phase-1_magicc7.1.0.beta-ipsl-cm6a-lr-r1i1p1f2_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-canesm5-r1i1p2f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-miroc6-r1i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-giss-e2-2-g-r1i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-bcc-esm1-r1i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-cesm2-r1i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-norcpm1-r1i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-canesm5-r10i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-noresm2-lm-r1i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-cnrm-cm6-1-hr-r1i1p1f2_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-giss-e2-1-h-r1i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-sam0-unicon-r1i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-cesm2-waccm-r1i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-e3sm-1-0-r1i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-ipsl-cm6a-lr-r10i1p1f1_v1-0-0.csv
rcmip_phase-1_magicc7.1.0.beta-fgoals

<IPython.core.display.Javascript object>